In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from mpl_toolkits.mplot3d import Axes3D
import random
import numpy as np
import os
import pandas as pd
import numpy as np
from ase.io import read
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Path setup
data_dir = "./"  # <-- Change this to your actual data path
atoms_train_dir = os.path.join(data_dir, "atoms", "train")
energies_csv = os.path.join(data_dir, "energies", "train.csv")

## Preliminaries

First, we import NumPy, PyTorch, and some utility modules.



In [3]:
import numpy as np
import torch
import time
import os

We will use scikit-learn to construct a linear model, so we import the
necessary modules. In addition, we need to compute distance matrices when
normalizing our input features, so we import `pdist` from `scipy.spatial`.



In [4]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

We then import the necessary functionality from Kymatio. First, we need the
PyTorch frontend of the 3D solid harmonic cattering transform.



In [5]:
from kymatio.torch import HarmonicScattering3D

The 3D transform doesn't compute the zeroth-order coefficients, so we need
to import `compute_integrals` to do this manually.



In [6]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

To generate the input 3D maps, we need to calculate sums of Gaussians, so we
import the function `generate_weighted_sum_of_gaussians`.



In [7]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

Finally, we import the utility functions that let us access the QM7 dataset
and the cache directories to store our results.



In [8]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

# Energies dataframe

In [9]:
# Read energies CSV
energies_df = pd.read_csv(energies_csv)
print("Loaded energy data:\n", energies_df.head().to_string(index=False))

Loaded energy data:
  id     energy
  1 -90.107880
  2 -69.927647
  3 -69.979214
  4 -73.402302
  5 -78.936724


# Positions and charges 

In [10]:
# === Extraction des features depuis les fichiers .xyz ===
def extract_features(xyz_path):
    atoms = read(xyz_path)
    positions = atoms.get_positions()
    charges = atoms.get_atomic_numbers()

    # masses = atoms.get_masses()
    # com = atoms.get_center_of_mass()
    # max_dist = np.linalg.norm(atoms.positions - com, axis=1).max()
    
    return {
        "positions":positions,
        "charges": charges
        # "masse_totale": masses.sum(),
        # "dist_centre_max": max_dist
    }

In [11]:
# Get all files and extract IDs
files = [f for f in os.listdir(atoms_train_dir) if f.endswith(".xyz")]

# Extract IDs and sort numerically
files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

data = []
for fname in tqdm(files):
    mol_id = int(fname.split("_")[1].split(".")[0])
    xyz_path = os.path.join(atoms_train_dir, fname)
    feats = extract_features(xyz_path)
    feats["id"] = mol_id  # Use the extracted ID
    data.append(feats)

100%|██████████| 6591/6591 [00:03<00:00, 1985.02it/s]


# Data preparation

In [14]:
data[0]

{'positions': array([[ 0.356436, -3.423619, -0.385554],
        [ 0.674042, -1.997613, -0.81716 ],
        [-0.160963, -0.953233, -0.067053],
        [ 0.15581 ,  0.481542, -0.509487],
        [-0.711321,  1.516257,  0.195147],
        [-0.257499,  2.171619,  1.453771],
        [-0.28779 ,  2.919863,  0.195085],
        [ 0.951293, -4.140487, -0.947535],
        [-0.693579, -3.657572, -0.54993 ],
        [ 0.567462, -3.568831,  0.672098],
        [ 0.501194, -1.89482 , -1.893711],
        [ 1.738769, -1.795955, -0.65804 ],
        [ 0.010933, -1.051982,  1.010204],
        [-1.225808, -1.156433, -0.22521 ],
        [ 0.012401,  0.563854, -1.594183],
        [ 1.213274,  0.70068 , -0.323745],
        [-1.790995,  1.380711,  0.058204],
        [ 0.723998,  1.91488 ,  1.857359],
        [-0.989227,  2.485891,  2.199616],
        [ 0.655782,  3.0113  , -0.188705]]),
 'charges': array([6, 6, 6, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'id': 1}

In [ ]:
result = {}

# Itérer sur chaque élément de data
for entry in data:
    # Utiliser l'id comme clé dans le dictionnaire result
    result[entry['id']] = {
        'charges': entry['charges'],
        'positions': entry['positions']
    }

In [17]:
result.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [115]:
# Initialiser les listes pour les positions et les charges
all_positions = []
all_charges = []

# Parcourir chaque dictionnaire dans la liste data
for entry in data:
    all_positions.extend(entry["positions"])
    all_charges.extend(entry["charges"])

# Créer le dictionnaire final
result = {
    "positions": all_positions,
    "charges": all_charges
}

In [56]:
# features_df['positions'] = features_df['positions'].apply(lambda x: [item for sublist in x for item in sublist])


In [83]:
pos.shape

(110053, 3)

In [134]:
len(result['positions'])

110053

In [135]:
pos = result['positions']
full_charges = np.array(result['charges'])

n_molecules = len(pos) #.shape[0]

In [136]:
full_charges

array([6, 6, 6, ..., 1, 1, 1])

In [137]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

In [140]:
pos

[array([ 0.356436, -3.423619, -0.385554]),
 array([ 0.674042, -1.997613, -0.81716 ]),
 array([-0.160963, -0.953233, -0.067053]),
 array([ 0.15581 ,  0.481542, -0.509487]),
 array([-0.711321,  1.516257,  0.195147]),
 array([-0.257499,  2.171619,  1.453771]),
 array([-0.28779 ,  2.919863,  0.195085]),
 array([ 0.951293, -4.140487, -0.947535]),
 array([-0.693579, -3.657572, -0.54993 ]),
 array([ 0.567462, -3.568831,  0.672098]),
 array([ 0.501194, -1.89482 , -1.893711]),
 array([ 1.738769, -1.795955, -0.65804 ]),
 array([ 0.010933, -1.051982,  1.010204]),
 array([-1.225808, -1.156433, -0.22521 ]),
 array([ 0.012401,  0.563854, -1.594183]),
 array([ 1.213274,  0.70068 , -0.323745]),
 array([-1.790995,  1.380711,  0.058204]),
 array([0.723998, 1.91488 , 1.857359]),
 array([-0.989227,  2.485891,  2.199616]),
 array([ 0.655782,  3.0113  , -0.188705]),
 array([-0.978257, -0.976201,  1.620806]),
 array([-1.408462,  0.179831,  0.72651 ]),
 array([-0.815131,  0.090689, -0.68896 ]),
 array([ 0.717

In [138]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i][:n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Scattering Transform



In [ ]:
M, N, O = 32, 32, 32

grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [ ]:
J = 1
L = 2
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

We then check whether a GPU is available, in which case we transfer our
scattering object there.



In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)

HarmonicScattering3D()

The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [ ]:
batch_size = 8
n_batches = int(np.ceil(n_molecules / batch_size))
print(n_batches)

896


We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [ ]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules,   "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None
for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
                    i + 1, int(((n_batches - i - 1) * dt) // 3600),
                    int((((n_batches - i - 1) * dt) // 60) % 60),
                    int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # Calculate the density map for the nuclear charges and transfer
    # to PyTorch.
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch)
    full_density_batch = full_density_batch.to(device).float()

    # Compute zeroth-order, first-order, and second-order scattering
    # coefficients of the nuclear charges.
    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch,
                                     integral_powers)
    full_scattering = scattering(full_density_batch)

    # Compute the map for valence charges.
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
            pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch)
    val_density_batch = val_density_batch.to(device).float()

    # Compute scattering coefficients for the valence charges.
    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch,
                                    integral_powers)
    val_scattering = scattering(val_density_batch)

    # Take the difference between nuclear and valence charges, then
    # compute the corresponding scattering coefficients.
    core_density_batch = full_density_batch - val_density_batch

    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch,
                                     integral_powers)
    core_scattering = scattering(core_density_batch)

    # Stack the nuclear, valence, and core coefficients into arrays
    # and append them to the output.
    batch_order_0 = torch.stack(
        (full_order_0, val_order_0, core_order_0), dim=-1)
    batch_orders_1_and_2 = torch.stack(
        (full_scattering, val_scattering, core_scattering), dim=-1)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)

Computing solid harmonic scattering coefficients of 7165 molecules from the QM7 database on GPU
sigma: 2.0, L: 2, J: 1, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:18:04]
Iteration 3 ETA: [00:15:50]
Iteration 4 ETA: [00:15:51]
Iteration 5 ETA: [00:16:00]
Iteration 6 ETA: [00:15:58]
Iteration 7 ETA: [00:15:49]
Iteration 8 ETA: [00:15:53]
Iteration 9 ETA: [00:16:05]
Iteration 10 ETA: [00:15:56]
Iteration 11 ETA: [00:15:56]
Iteration 12 ETA: [00:15:45]
Iteration 13 ETA: [00:15:52]
Iteration 14 ETA: [00:15:53]
Iteration 15 ETA: [00:15:49]
Iteration 16 ETA: [00:15:39]
Iteration 17 ETA: [00:15:46]
Iteration 18 ETA: [00:15:54]
Iteration 19 ETA: [00:15:51]
Iteration 20 ETA: [00:15:51]
Iteration 21 ETA: [00:15:44]
Iteration 22 ETA: [00:15:42]
Iteration 23 ETA: [00:16:02]
Iteration 24 ETA: [00:15:47]
Iteration 25 ETA: [00:15:38]
Iteration 26 ETA: [00:15:40]
Iteration 27 ETA: [00:15:37]
Iteration 28 ETA: [00:15:38]
Iteration 29 ETA: [00:15:59]
Iteration 30 ETA: [

KeyboardInterrupt: 

Concatenate the batch outputs and transfer to NumPy.



In [ ]:
order_0 = torch.cat(order_0, dim=0)
orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()